In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth


In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load Mistral-7B 4-bit
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Apply LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.7: Fast Mistral patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com

model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2025.9.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
import json
from datasets import Dataset

# --- Training set ---
with open("train.json") as f:
    train_data = json.load(f)

with open("train_tables.json") as f:
    train_tables = json.load(f)

# --- Validation set (dev + tied append) ---
with open("dev.json") as f:
    dev_data = json.load(f)

with open("dev_tied_append.json") as f:
    dev_tied_data = json.load(f)

with open("dev_tables.json") as f:
    dev_tables = json.load(f)

# Merge dev + dev_tied_append
dev_combined = dev_data + dev_tied_data

print("Train examples:", len(train_data))
print("Dev examples:", len(dev_combined))


Train examples: 9428
Dev examples: 1576


In [4]:
def build_schema_lookup(tables):
    schema_lookup = {}
    for table in tables:
        table_id = table["db_id"]
        columns = [f"{col[1]} ({col[0]})"
                   for col in table["column_names_original"] if col[1] != "*"]
        full_schema = f"Tables: {', '.join(table['table_names_original'])}. Columns: {', '.join(columns)}"
        schema_lookup[table_id] = full_schema
    return schema_lookup

train_schema_lookup = build_schema_lookup(train_tables)
dev_schema_lookup = build_schema_lookup(dev_tables)


In [5]:
def format_dataset(data, schema_lookup):
    formatted = []
    for item in data:
        db_id = item["db_id"]
        question = item["question"]
        sql = item["SQL"]
        schema = schema_lookup.get(db_id, "No schema found.")
        text = f"### Instruction:\nGiven the schema: {schema}\nTranslate the following question into SQL:\n{question}\n\n### Response:\n{sql}"
        formatted.append({"text": text})
    return formatted

train_formatted = format_dataset(train_data, train_schema_lookup)
dev_formatted = format_dataset(dev_combined, dev_schema_lookup)

hf_train = Dataset.from_list(train_formatted)
hf_dev = Dataset.from_list(dev_formatted)

print("Train size:", len(hf_train))
print("Dev size:", len(hf_dev))
print(hf_train[0])


Train size: 9428
Dev size: 1576
{'text': '### Instruction:\nGiven the schema: Tables: lists, movies, ratings_users, lists_users, ratings. Columns: user_id (0), list_id (0), list_title (0), list_movie_number (0), list_update_timestamp_utc (0), list_creation_timestamp_utc (0), list_followers (0), list_url (0), list_comments (0), list_description (0), list_cover_image_url (0), list_first_image_url (0), list_second_image_url (0), list_third_image_url (0), movie_id (1), movie_title (1), movie_release_year (1), movie_url (1), movie_title_language (1), movie_popularity (1), movie_image_url (1), director_id (1), director_name (1), director_url (1), user_id (2), rating_date_utc (2), user_trialist (2), user_subscriber (2), user_avatar_image_url (2), user_cover_image_url (2), user_eligible_for_trial (2), user_has_payment_method (2), user_id (3), list_id (3), list_update_date_utc (3), list_creation_date_utc (3), user_trialist (3), user_subscriber (3), user_avatar_image_url (3), user_cover_image_ur

In [9]:
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer

dataset_text_field = "text"

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = hf_train,
    eval_dataset = hf_dev,
    dataset_text_field = dataset_text_field,
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model),
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 25,
        learning_rate = 1e-4,
        fp16 = True,
        logging_steps = 5,
        eval_strategy = "steps",
        eval_steps = 10,
        save_steps = 10,
        output_dir = "outputs",
        report_to = "none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/9428 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1576 [00:00<?, ? examples/s]

In [10]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,428 | Num Epochs = 1 | Total steps = 25
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Step,Training Loss,Validation Loss
10,0.096600,1.214215
20,0.011100,1.367272


Unsloth: Not an error, but MistralForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=25, training_loss=0.186294609233737, metrics={'train_runtime': 2019.6991, 'train_samples_per_second': 0.099, 'train_steps_per_second': 0.012, 'total_flos': 5649412060938240.0, 'train_loss': 0.186294609233737, 'epoch': 0.021213406873143825})

In [11]:
# Save to a directory (e.g., "fine-tuned-mistral")
save_path = "fine-tuned-mistral"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('fine-tuned-mistral/tokenizer_config.json',
 'fine-tuned-mistral/special_tokens_map.json',
 'fine-tuned-mistral/tokenizer.model',
 'fine-tuned-mistral/added_tokens.json',
 'fine-tuned-mistral/tokenizer.json')

In [12]:
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l

In [13]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 139.8 MB/s eta 0:00:00


In [22]:
!ngrok config add-authtoken 33CPxN7YfOrNiC6gDQWv2rM7LEi_2Q36iGpiXACw5m73poxTx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Terminate any existing tunnels
ngrok.kill()

# Start a new tunnel
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run /content/app.py

Streamlit app is live at: NgrokTunnel: "https://2e3d7bcda07f.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.168.212.77:8501



In [16]:
!pip install pyngrok


In [17]:
!zip -r fine-tuned-mistral.zip fine-tuned-mistral

  adding: fine-tuned-mistral/ (stored 0%)
  adding: fine-tuned-mistral/adapter_model.safetensors (deflated 8%)
  adding: fine-tuned-mistral/README.md (deflated 65%)
  adding: fine-tuned-mistral/tokenizer.model (deflated 55%)
  adding: fine-tuned-mistral/adapter_config.json (deflated 58%)
  adding: fine-tuned-mistral/tokenizer_config.json (deflated 68%)
  adding: fine-tuned-mistral/special_tokens_map.json (deflated 79%)
  adding: fine-tuned-mistral/tokenizer.json (deflated 85%)


In [18]:
from google.colab import files
files.download("fine-tuned-mistral.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
SELECT 
    c.candidate_id,
    c.full_name,
    r.recruiter_name,
    j.job_title
FROM 
    candidates c
JOIN 
    applications a 
        ON c.candidate_id = a.candidate_id
JOIN 
    jobs j 
        ON a.job_id = j.job_id
JOIN 
    recruiters r 
        ON j.recruiter_id = r.recruiter_id
JOIN 
    (
        SELECT 
            candidate_id,
            job_id,
            COUNT(*) AS total_rounds,
            SUM(CASE WHEN status = 'cleared' THEN 1 ELSE 0 END) AS cleared_rounds
        FROM interviews
        GROUP BY candidate_id, job_id
        HAVING COUNT(*) = SUM(CASE WHEN status = 'cleared' THEN 1 ELSE 0 END)
    ) AS cleared_all_rounds
        ON cleared_all_rounds.candidate_id = c.candidate_id
       AND cleared_all_rounds.job_id = j.job_id
LEFT JOIN 
    offers o
        ON o.candidate_id = c.candidate_id
       AND o.job_id = j.job_id
WHERE 
    o.offer_id IS NULL;     -- meaning no offer given
